# Một số thủ thuật pandas cho panel data (bài 2)

### Bs. Lê Ngọc Khả Nhi

# Giới thiệu

Đây là phần tiếp nối của bài thực hành trước, chúng ta sẽ tiếp tục khám phá một số thủ thuật khác mà thư viện pandas cung cấp, giúp thực hiện các phân tích thống kê trên panel data.

Chúng ta tiếp tục làm việc trên bộ dữ liệu Mayo Clinic Primary Biliary Cirrhosis (PBC). Lần trước ta dừng lại ở câu hỏi thứ 12, trong bài này ta sẽ giải quyết 6 câu hỏi khác thú vị không kém

In [1]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv('PBC2.csv', sep = ';')

df.head(5)

,id,years,status,drug,age,sex,year,ascites,hepatomegaly,spiders,edema,serBilir,serChol,albumin,alkaline,SGOT,platelets,prothrombin,histologic,status2
0,1,1.095170,dead,D-penicil,58.766838,female,0.000000,Yes,Yes,Yes,edema despite diuretics,14.5,261.0,2.60,1718.0,138.0,190.0,12.2,4,1
1,1,1.095170,dead,D-penicil,58.766838,female,0.525682,Yes,Yes,Yes,edema despite diuretics,21.3,NaN,2.94,1612.0,6.2,183.0,11.2,4,1
2,2,14.152338,alive,D-penicil,56.447815,female,0.000000,No,Yes,Yes,No edema,1.1,302.0,4.14,7395.0,113.5,221.0,10.6,3,0
3,2,14.152338,alive,D-penicil,56.447815,female,0.498302,No,Yes,Yes,No edema,0.8,NaN,3.60,2107.0,139.5,188.0,11.0,3,0
4,2,14.152338,alive,D-penicil,56.447815,female,0.999343,No,Yes,Yes,No edema,1.0,NaN,3.55,1711.0,144.2,161.0,11.6,3,0


# Câu hỏi 13: Xác định tỉ lệ phân bố giới tính trong 2 phác đồ điều trị x 2 kết cục lâm sàng

Nếu đã xem bài trước, các bạn có thể nhận ra câu hỏi này chính là phép đếm tần suất cá thể như trong câu hỏi 3, bạn có thể giải quyết theo cùng một cách, đầu tiên ta áp dụng groupby cho tổ hợp 3 biến drug, status và sex, sau đó đếm id đã được rút gọn qua hàm unique. Bạn có thể dùng agg hay apply đều được.

In [29]:
df.groupby(['drug','status','sex'])['id'].unique().agg(len)

drug       status        sex   
D-penicil  alive         female    72
                         male       3
           dead          female    56
                         male      15
           transplanted  female     9
                         male       3
placebo    alive         female    64
                         male       4
           dead          female    58
                         male      11
           transplanted  female    17
Name: id, dtype: int64

Tuy nhiên trong bài này Nhi sẽ giới thiệu một cách làm khác thông qua method pivot_table mà pandas cung cấp sẵn. pivot_table là công cụ rất mạnh và phổ quát cho phép tạo ra mọi cấu trúc bảng chéo mà ta muốn.

Trong thí dụ này, ta dùng pivot_table với values là id, index (hàng) là tổ hợp drug, status và sex, và aggfunction là 1 hàm lambda thực hiện phép đếm tần suất rút gọn cho id.

Kết quả của pivot_table là 1 dataframe, khác với 

In [9]:
pd.pivot_table(df, values = ['id'], index=['drug', 'status','sex'],
                aggfunc=lambda x: len(x.unique()))

id
drug      status       sex       
D-penicil alive        female  72
                       male     3
          dead         female  56
                       male    15
          transplanted female   9
                       male     3
placebo   alive        female  64
                       male     4
          dead         female  58
                       male    11
          transplanted female  17

# Câu hỏi 14: Bổ túc dữ liệu thiếu bằng giá trị gần nhất cho mỗi bệnh nhân

Như ta thấy, trong dataframe df có vài biến số với nội dung không hoàn chỉnh, bị thiếu sót dữ liệu, như các biến như ascites, hepatomegaly, serChol...

In [30]:
df.isna().sum()

id                0
years             0
status            0
drug              0
age               0
sex               0
year              0
ascites          60
hepatomegaly     61
spiders          58
edema             0
serBilir          0
serChol         821
albumin           0
alkaline         60
SGOT              0
platelets        73
prothrombin       0
histologic        0
status2           0
dtype: int64

Một kỹ thuật có thể áp dụng để giải quyết vấn đề thiếu sót dữ liệu trong panel data, đó là bổ túc dữ liệu bị thiếu sót tại 1 thời điểm bằng giá trị gần nhất (của lần tái khám ngay trước đó), nhưng phải áp dụng riêng biệt cho mỗi bệnh nhân. Ta có thể triển khai giải pháp này như sau:

Đầu tiên Nhi viết 1 hàm để bổ sung dữ liệu bằng phương pháp "forward fill"

In [48]:
def fillna_by_grp(x: pd.DataFrame) -> pd.DataFrame:
    x.fillna(method = 'ffill', inplace = True)
    return x

Sau đó dùng groupby và aply để áp dụng hàm bổ túc dữ liệu này cho 2 cột ascites, hepatomegaly, và gán trở lại vào dữ liệu gốc.

In [50]:
df.loc[:,['ascites','hepatomegaly']] = df.groupby('id')[['ascites','hepatomegaly']].apply(lambda x: fillna_by_grp(x))

Kết quả là toàn bộ 60 missing value của 2 cột đã được lấp đầy:

In [51]:
df.isna().sum()

id                0
years             0
status            0
drug              0
age               0
sex               0
year              0
ascites           0
hepatomegaly      0
spiders          58
edema             0
serBilir          0
serChol         821
albumin           0
alkaline         60
SGOT              0
platelets        73
prothrombin       0
histologic        0
status2           0
dtype: int64

# Câu hỏi 15: Tạo bảng thống kê mô tả theo ý thích

Yêu cầu của câu hỏi 15 như sau: 

Hãy tạo 1 bảng thống kê mô tả có nội dung: Mỗi hàng là một biến số liên tục (cho toàn bộ biến liên tục trong df). Tên mỗi biến bằng tiếng Việt. 2 cột thể hiện giá trị trung bình ± SD cho mỗi biến trong 2 phân nhóm can thiệp. Giá trị mean và (sd) chính xác đến 2 số lẻ.

Ta có thể thỏa mãn tất cả yêu cầu trên bằng pandas không ? Hoàn toàn có thể...

Đầu tiên, Nhi tạo 1 dictionary để dịch tên cột trong df thành tiếng Việt:

In [53]:
viet_lst = ['Mã số',
             'Thời gian NC',
             'Kết cục lâm sàng',
             'Nhóm can thiệp',
             'Tuổi',
             'Giới tính',
             'Thời gian',
             'Báng bụng',
             'Gan to',
             'Sao mạch',
             'Phù',
             'Bilirubin',
             'Cholesterol',
             'Albumin',
             'Alkaline',
             'SGOT',
             'Tiểu cầu',
             'Thời gian Prothrombin',
             'Giai đoạn bệnh',
             'Kết cục nhị phân']

ev_dict = {k:v for k,v in zip(df.columns, viet_lst)}

ev_dict

{'id': 'Mã số',
 'years': 'Thời gian NC',
 'status': 'Kết cục lâm sàng',
 'drug': 'Nhóm can thiệp',
 'age': 'Tuổi',
 'sex': 'Giới tính',
 'year': 'Thời gian',
 'ascites': 'Báng bụng',
 'hepatomegaly': 'Gan to',
 'spiders': 'Sao mạch',
 'edema': 'Phù',
 'serBilir': 'Bilirubin',
 'serChol': 'Cholesterol',
 'albumin': 'Albumin',
 'alkaline': 'Alkaline',
 'SGOT': 'SGOT',
 'platelets': 'Tiểu cầu',
 'prothrombin': 'Thời gian Prothrombin',
 'histologic': 'Giai đoạn bệnh',
 'status2': 'Kết cục nhị phân'}

Sau đó, chỉ cần 2 dòng code là có thể tạo ra bảng thống kê mô tả nói trên, ta cũng kết hợp groupby và agg(), i dung mean ± sd, số lẻ, vv được tạo ra bằng 1 hàm lambda và f-string mà Python cung cấp.

In [54]:
out_df = df.groupby(['drug'])[df.columns[df.dtypes.eq('float64')]].agg(lambda x: f"{x.mean():.2f} ± {x.std():.2f}").T

out_df.index = out_df.index.map(ev_dict)

out_df

drug,D-penicil,placebo
Thời gian NC,8.06 ± 3.50,8.04 ± 3.46
Tuổi,50.83 ± 10.57,47.68 ± 9.26
Thời gian,3.20 ± 3.14,3.07 ± 3.05
Bilirubin,3.61 ± 5.22,3.73 ± 5.52
Cholesterol,322.05 ± 163.51,318.88 ± 170.02
Albumin,3.40 ± 0.50,3.38 ± 0.51
Alkaline,1334.17 ± 1186.06,1430.01 ± 1203.90
SGOT,115.65 ± 87.81,129.77 ± 66.95
Tiểu cầu,231.45 ± 103.05,235.94 ± 91.88
Thời gian Prothrombin,11.00 ± 1.59,10.99 ± 1.35


# Câu hỏi 16: Tạo bảng chéo với nội dung tùy biến

Câu 15 trên chưa phải là cấu trúc phức tạp nhất mà ta có thể làm với pandas, tiếp theo ta sẽ làm một bảng thống kê mô tả phức tạp hơn nữa với yêu cầu như sau:

Mỗi hàng là một phân nhóm tổ hợp giữa 2 phác đồ điều trị và 3 kết cục lâm sàng,
Có 3 cột, cột SGOT trình bày giá trị trung vị (min - max) của SGOT cho mỗi phân nhóm,
Cột Albumin trình bày mean ± sd cho Albumin cho mỗi phân nhóm,
Cột n patients trình bày tần suất bệnh nhân cho mỗi phân nhóm.

Thách thức được chấp nhận, công cụ ta sẽ dùng chính là pivot_table, vì nó cho phép áp dụng 3 hàm aggfunc riêng biệt cho 3 biến trong 1 dictionary:

In [65]:
out_df = pd.pivot_table(df, values = ['id','albumin','SGOT'], 
               index=['drug', 'status'],
               aggfunc={'id': lambda x: len(x.unique()),
                        'albumin': lambda x: f"{np.mean(x):.2f} ± {np.std(x):.2f}",
                        'SGOT': lambda x: f"{np.median(x):.2f} ({np.min(x):.2f} - {np.max(x):.2f})"
                       })

out_df.columns = ['SGOT', 'Albumin','n patients']

out_df

SGOT      Albumin  n patients
drug      status                                                        
D-penicil alive          76.00 (21.00 - 918.00)  3.54 ± 0.43          75
          dead          127.10 (6.20 - 1205.00)  3.19 ± 0.53          71
          transplanted  156.60 (57.00 - 266.00)  3.36 ± 0.46          12
placebo   alive          96.10 (21.70 - 457.30)  3.50 ± 0.45          68
          dead          139.75 (28.40 - 655.70)  3.22 ± 0.57          69
          transplanted  146.15 (54.30 - 336.00)  3.37 ± 0.38          17

# Câu hỏi 17: Tính sự thay đổi tương đối (%) so với baseline

Hãy hoán chuyển 2 biến SGOT và Albumin thành 2 biến khác có tên là SGOT_change và Albumin_change, khảo sát sự thay đổi tương đối (tỉ lệ phần trăm) so với giá trị ở thời điểm baseline của mỗi bệnh nhân

Câu hỏi này gần giống với câu 8 trong bài trước, ta có thể giải quyết tương tự với 1 hàm rời, sau đó dùng groupby và transform với hàm rời này:

In [66]:
def pct_change_from_baseline(x):
    return x.apply(lambda v: (v-x.values[0])/x.values[0])

df['SGOT_change'] = df.groupby('id')[['SGOT']].transform(pct_change_from_baseline)
df['Albumin_change'] = df.groupby('id')[['albumin']].transform(pct_change_from_baseline)

df[['SGOT_change', 'Albumin_change']]

,SGOT_change,Albumin_change
0,0.000000,0.000000
1,-0.955072,0.130769
2,0.000000,0.000000
3,0.229075,-0.130435
4,0.270485,-0.142512
...,...,...
1940,0.000000,0.000000
1941,-0.088235,-0.155673
1942,0.220588,-0.060686
1943,0.272059,-0.118734


# Câu hỏi 18: Phân tích tương quan cho từng phân nhóm

Hãy phân tích tương quan về tỉ lệ thay đổi tại lần khám sau cùng so với baseline, giữa Albumin và SGOT, riêng cho 2 kết cục lâm sàng

Đây là 1 câu hỏi phức tạp, chú ý rằng ta chỉ xét giá trị của lần khám sau cùng, do đó bước 1 ta cần áp dụng groupby cho id, sau đó dùng apply với 1 hàm lambda và dùng dataframe iloc để lấy giá trị sau cùng, ta lưu lại thành dataframe mới là rc_df

In [67]:
rc_df = df.groupby(['id'])[['status','SGOT_change','Albumin_change']].apply(lambda x: x.iloc[-1])

rc_df

,status,SGOT_change,Albumin_change
id,,,
1,dead,-0.955072,0.130769
2,alive,-0.224670,-0.355072
3,dead,-0.080125,-0.066092
4,dead,1.174917,-0.279528
5,transplanted,0.378092,-0.407932
...,...,...,...
308,alive,0.131868,-0.131285
309,alive,0.285714,0.174545
310,alive,-0.619883,-0.047761


Bước 2, ta áp dụng groupby cho status trên rc_df, sau đó apply hàm lambda và method corr của 1 dataframe để tạo ra 3 correlation matrix cho 3 kết cục lâm sàng

In [68]:
rc_df.groupby('status')[['SGOT_change','Albumin_change']].apply(lambda x: x.corr(method='pearson'))

SGOT_change  Albumin_change
status                                                  
alive        SGOT_change        1.000000       -0.148978
             Albumin_change    -0.148978        1.000000
dead         SGOT_change        1.000000       -0.216481
             Albumin_change    -0.216481        1.000000
transplanted SGOT_change        1.000000       -0.199159
             Albumin_change    -0.199159        1.000000

Đây chỉ mới là giá trị hệ số tương quan, vậy nếu muốn có p values thì sao ? Chuyện nhỏ ! ta có thể làm như sau:

Viết 1 hàm rời để lấy p value từ method pearsonr của scipy.stats, sau đó dùng hàm này như method trong hàm x.corr() như trên

In [69]:
from scipy.stats import pearsonr

def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

rc_df.groupby('status')[['SGOT_change','Albumin_change']].apply(lambda x: x.corr(method=pearsonr_pval))

SGOT_change  Albumin_change
status                                                  
alive        SGOT_change        1.000000        0.075766
             Albumin_change     0.075766        1.000000
dead         SGOT_change        1.000000        0.010199
             Albumin_change     0.010199        1.000000
transplanted SGOT_change        1.000000        0.300320
             Albumin_change     0.300320        1.000000

# Tổng kết

Qua 18 câu hỏi trong 2 bài thực hành, chắc các bạn đã có thể tự tin khi thao tác trên dữ liệu hỗn hợp (panel data). Chúc các bạn thực hành vui.